<a href="https://colab.research.google.com/github/NathanielRose/eeg-294/blob/master/Gumpy-Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Testing for EEG | ML 294**

In [1]:
!pip install numpy scipy matplotlib

In [2]:
!git clone https://github.com/NathanielRose/eeg-294.git

Cloning into 'eeg-294'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 107 (delta 27), reused 81 (delta 9), pack-reused 0
Receiving objects: 100% (107/107), 630.50 KiB | 14.33 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [3]:
!pip install git+git://github.com/gumpy-bci/gumpy.git

  Cloning git://github.com/gumpy-bci/gumpy.git to /tmp/pip-req-build-tl3xp_ge
  Running command git clone -q git://github.com/gumpy-bci/gumpy.git /tmp/pip-req-build-tl3xp_ge
  Created wheel for gumpy: filename=gumpy-0.5.0-cp36-none-any.whl size=32999 sha256=22971b70c12c230a5f125332b5370d5ced42c6e3c3d867f917b875138aab08d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-wba1lynb/wheels/cf/ee/4d/dfa04a0d04ffb1ca36f436f3fef4817e9e23bfec03dd772913
Successfully built gumpy


In [4]:
from __future__ import print_function
import os; os.environ["THEANO_FLAGS"] = "device=gpu0"
import os.path
from datetime import datetime
import sys
sys.path.append('../../gumpy')

import numpy as np
import scipy.io
import matplotlib.pyplot as plt

sys.path.append('eeg-294/gumpy/gumpy')
import gumpy

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
sys.path.append('eeg-294/gumpy-deeplearning')
import models

Using TensorFlow backend.


In [6]:
sys.path.append('eeg-294/gumpy-deeplearning/models/')
import utils

ImportError: ignored

In [0]:
DEBUG = True
CLASS_COUNT = 2
DROPOUT = 0.2   # dropout rate in float

# parameters for filtering data
FS = 250
LOWCUT = 2
HIGHCUT = 60
ANTI_DRIFT = 0.5
CUTOFF = 50.0 # freq to be removed from signal (Hz) for notch filter
Q = 30.0  # quality factor for notch filter 
W0 = CUTOFF/(FS/2)
AXIS = 0

#set random seed
SEED = 42
KFOLD = 5

In [0]:
https://drive.google.com/open?id=1DS64UOrt9wzswke5wiFtat94-s1YcRmb

In [35]:
%%bash

export fileid=1DS64UOrt9wzswke5wiFtat94-s1YcRmb
export filename=grazdata.zip


wget --save-cookies cookies.txt 'https://docs.google.com/uc?export=download&id='$fileid -O- \
     | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1/p' > confirm.txt

wget --load-cookies cookies.txt -O $filename \
     'https://docs.google.com/uc?export=download&id='$fileid'&confirm='$(<confirm.txt)

--2019-10-30 03:02:48--  https://docs.google.com/uc?export=download&id=1DS64UOrt9wzswke5wiFtat94-s1YcRmb
Resolving docs.google.com (docs.google.com)... 173.194.213.139, 173.194.213.113, 173.194.213.100, ...
Connecting to docs.google.com (docs.google.com)|173.194.213.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘STDOUT’

     0K ...                                                    17.0M=0s

2019-10-30 03:02:49 (17.0 MB/s) - written to stdout [3248]

--2019-10-30 03:02:49--  https://docs.google.com/uc?export=download&id=1DS64UOrt9wzswke5wiFtat94-s1YcRmb&confirm=SFwo
Resolving docs.google.com (docs.google.com)... 173.194.213.113, 173.194.213.139, 173.194.213.101, ...
Connecting to docs.google.com (docs.google.com)|173.194.213.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/tn49d1j75u

In [37]:
!unzip grazdata.zip

Archive:  grazdata.zip
mapname:  conversion of  failed
 extracting: B03T.mat                
 extracting: B05E.mat                
 extracting: B04T.mat                
 extracting: B01E.mat                
 extracting: B04E.mat                
 extracting: B06T.mat                
 extracting: B08T.mat                
 extracting: B07T.mat                
 extracting: B02T.mat                
 extracting: B03E.mat                
 extracting: B09T.mat                
 extracting: B05T.mat                
 extracting: B09E.mat                
 extracting: B01T.mat                
 extracting: B06E.mat                
 extracting: B07E.mat                
 extracting: B08E.mat                
 extracting: B02E.mat                


In [29]:
!ls

 eeg-294   FILENAME  'open?id=1DS64UOrt9wzswke5wiFtat94-s1YcRmb'   sample_data


In [19]:
!unzip grazdata

Archive:  grazdata
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
Archive:  grazdata.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of grazdata or
        grazdata.zip, and cannot find grazdata.ZIP, period.


In [0]:
sys.path.append('eeg-294/gumpy/gumpy')
# specify the location of the GrazB datasets
data_dir = '../../Data/Graz'
subject = 'B01'

# initialize the data-structure, but do _not_ load the data yet
grazb_data = gumpy.data.GrazB(data_dir, subject)

# now that the dataset is setup, we can load the data. This will be handled from within the utils function, 
# which will first load the data and subsequently filter it using a notch and a bandpass filter.
# the utility function will then return the training data.
x_train, y_train = utils.load_preprocess_data(grazb_data, True, LOWCUT, HIGHCUT, W0, Q, ANTI_DRIFT, CLASS_COUNT, CUTOFF, AXIS, FS)